In [1]:
import reskit as rk

# Simple Down Scaling

* RESKit will downscale low-spatial resolution weather data to a higher spatial resolution using a simplistic 'long run average' approach
* When downscaling wind speeds, comparisons are made against the Global Wind Atlas (1 km resolution)
* When downscaling irradiance values, comparisons are made against the Global Solar Atlas (1 km resolution)

In [2]:
# Create a weather source, and load data
src = rk.weather.sources.MerraSource(rk._TEST_DATA_["weather_data"], bounds=[5,49,7,52], verbose=False)
src.loadWindSpeed()

# Get an adjusted time series for a specific location

In [3]:
# Extract raw wind speed and GHI data
location = (6.0,50.5)

windspeeds = src.get("windspeed", locations=location)

In [5]:
# "rk._TEST_DATA_['gwa50-like.tif']" points to a local Gobal Wind Atlas file, this will need to be downloaded 
#   by the user for production use
#   - It is important to keep in mind the height-context of the weather data in relation to glabal wind atlas 
#     data (offered at 50m or 100m)
#
# "src.LONG_RUN_AVERAGE_50M_SOURCE" points to a raster file with the average wind speed value in each cell of a weather dataset
#   - This must be pre-computed
#   - For the MERRA2 data set over Europe, a raster file is automatically included. If you would like to use a different dataset 
#     or a different regional scope, you will need to compute this yourself

adjusted_wind_speeds = rk.weather.windutil.adjustLraToGwa(windspeeds, location, 
                                                          gwa=rk._TEST_DATA_['gwa50-like.tif'], 
                                                          longRunAverage=src.LONG_RUN_AVERAGE_50M_SOURCE)
adjusted_wind_speeds.head()

2015-01-01 00:30:00.000013+00:00    3.742758
2015-01-01 01:30:00+00:00           3.651281
2015-01-01 02:30:00.000026+00:00    3.539215
2015-01-01 03:30:00.000013+00:00    3.453292
2015-01-01 04:30:00+00:00           3.603335
Name: (6.00000,50.50000), dtype: float32

# Get adjusted time series for multiple locations

In [6]:
# Extract raw wind speed and GHI data
locations = [(6.25,51.), (6.50,51.), (6.25,50.75)]

windspeeds = src.get("windspeed", locations=locations)

windspeeds.head()

,"(6.25000,51.00000)","(6.50000,51.00000)","(6.25000,50.75000)"
2015-01-01 00:30:00.000013+00:00,6.117667,6.117667,6.117667
2015-01-01 01:30:00+00:00,6.147689,6.147689,6.147689
2015-01-01 02:30:00.000026+00:00,6.101908,6.101908,6.101908
2015-01-01 03:30:00.000013+00:00,5.905268,5.905268,5.905268
2015-01-01 04:30:00+00:00,5.990820,5.990820,5.990820


In [7]:
adjusted_wind_speeds = rk.weather.windutil.adjustLraToGwa(windspeeds, locations, 
                                                          gwa=rk._TEST_DATA_['gwa50-like.tif'], 
                                                          longRunAverage=src.LONG_RUN_AVERAGE_50M_SOURCE)
adjusted_wind_speeds.head()

,"(6.25000,51.00000)","(6.50000,51.00000)","(6.25000,50.75000)"
2015-01-01 00:30:00.000013+00:00,4.851627,4.991501,3.720944
2015-01-01 01:30:00+00:00,4.875437,5.015997,3.739205
2015-01-01 02:30:00.000026+00:00,4.839130,4.978643,3.711359
2015-01-01 03:30:00.000013+00:00,4.683184,4.818202,3.591757
2015-01-01 04:30:00+00:00,4.751032,4.888006,3.643792


# Employ a 'bl-linear' interpolation scheme

In [8]:
# using a bilienar interpolation scheme will create a smoother outcome (geospatially-speaking)
# Extract raw wind speed and GHI data
locations = [(6.25,51.), (6.50,51.), (6.25,50.75)]

windspeeds = src.get("windspeed", locations=locations, interpolation="bilinear")

windspeeds.head()

,"(6.25000,51.00000)","(6.50000,51.00000)","(6.25000,50.75000)"
2015-01-01 00:30:00.000013+00:00,6.117667,5.675237,5.787824
2015-01-01 01:30:00+00:00,6.147689,5.675423,5.736136
2015-01-01 02:30:00.000026+00:00,6.101908,5.648852,5.631533
2015-01-01 03:30:00.000013+00:00,5.905268,5.506286,5.470564
2015-01-01 04:30:00+00:00,5.990820,5.626991,5.622742


In [9]:
adjusted_wind_speeds = rk.weather.windutil.adjustLraToGwa(windspeeds, locations, interpolation="bilinear",
                                                          gwa=rk._TEST_DATA_['gwa50-like.tif'], 
                                                          longRunAverage=src.LONG_RUN_AVERAGE_50M_SOURCE)
adjusted_wind_speeds.head()

,"(6.25000,51.00000)","(6.50000,51.00000)","(6.25000,50.75000)"
2015-01-01 00:30:00.000013+00:00,4.851628,4.630516,3.520324
2015-01-01 01:30:00+00:00,4.875437,4.630668,3.488886
2015-01-01 02:30:00.000026+00:00,4.839130,4.608988,3.425264
2015-01-01 03:30:00.000013+00:00,4.683184,4.492666,3.327357
2015-01-01 04:30:00+00:00,4.751032,4.591151,3.419916
